# Welcome to Modal notebooks!

Write Python code and collaborate in real time. Your code runs in Modal's
**serverless cloud**, and anyone in the same workspace can join.

This notebook comes with some common Python libraries installed. Run
cells with `Shift+Enter`.

Assignment 1

In [3]:
%uv pip install datasets

Using Python 3.12.6 environment at: /usr/local
Resolved 36 packages in 173ms
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
⠙ Preparing packages... (0/5)
dill       ------------------------------ 14.92 KiB/116.86 KiB
⠙ Preparing packages... (0/5)
dill       ------------------------------ 14.92 KiB/116.86 KiB
multiprocess ------------------------------     0 B/146.76 KiB
⠙ Preparing packages... (0/5)
dill       ------------------------------ 14.92 KiB/116.86 KiB
multiprocess ------------------------------     0 B/146.76 KiB
datasets   ------------------------------     0 B/499.60 KiB
⠙ Preparing packages... (0/5)
dill       ------------------------------ 14.92 KiB/116.86 KiB
multiprocess ------------------------------     0 B/146.76 KiB
datasets   ------------------------------     0 B/499.60 KiB
⠙ Preparing packages... (0/5)
dill       ------------------------------ 14.92 KiB/116.86 KiB
multiprocess ------------------------------     0 B/146.76 KiB
datasets   -------------

In [4]:
import os
import hashlib
from typing import List, Dict

from datasets import load_dataset
from transformers import AutoTokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import re

DATA_DIR = "./processed_data"
os.makedirs(DATA_DIR, exist_ok=True)

TOKENIZER_NAME = "gpt2"
BLOCK_SIZE = 512
MIN_WORDS = 50

TARGET_SIZE_GB = 1.5
TARGET_CHARS_TOTAL = int(TARGET_SIZE_GB * 1e9)

SOURCE_FRACTIONS = {
    "wikipedia": 0.4,
    "fineweb": 0.4,
    "ag_news": 0.2,
}

SAMPLE_PT_PATH = os.path.join(DATA_DIR, "sample_dataset.pt")


In [5]:
def load_streaming_corpora():

    print("Loading Wikipedia (wikimedia/wikipedia, streaming)...")
    wiki = load_dataset(
        "wikimedia/wikipedia",
        "20231101.en",
        split="train",
        streaming=True,
    )

    print("Loading FineWeb-Edu (HuggingFaceFW/fineweb-edu, streaming)...")
    fineweb = load_dataset(
        "HuggingFaceFW/fineweb-edu",
        name="sample-10BT",
        split="train",
        streaming=True,
    )

    print("Loading AG News (SetFit/ag_news, regular load)...")
    ag = load_dataset("SetFit/ag_news", split="train")
    ag_iter = ag.to_iterable_dataset()

    return {
        "wikipedia": wiki,
        "fineweb": fineweb,
        "ag_news": ag_iter,
    }

streaming_corpora = load_streaming_corpora()
streaming_corpora



Loading Wikipedia (wikimedia/wikipedia, streaming)...


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Loading FineWeb-Edu (HuggingFaceFW/fineweb-edu, streaming)...


README.md: 0.00B [00:00, ?B/s]

...

train.jsonl:   0%|          | 0.00/33.8M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

{'wikipedia': IterableDataset({
     features: ['id', 'url', 'title', 'text'],
     num_shards: 41
 }),
 'fineweb': IterableDataset({
     features: ['text', 'id', 'dump', 'url', 'file_path', 'language', 'language_score', 'token_count', 'score', 'int_score'],
     num_shards: 14
 }),
 'ag_news': IterableDataset({
     features: ['text', 'label', 'label_text'],
     num_shards: 1
 })}

In [6]:
whitespace_re = re.compile(r"\s+")
def clean_text(text: str) -> str:
    """
    Basic cleaning:
    - lowercase
    - remove simple HTML-like tags
    - normalize whitespace
    """
    if text is None:
        return ""

    text = text.lower()
    text = re.sub(r"<[^>]+>", " ", text)
    text = whitespace_re.sub(" ", text)
    text = text.strip()
    return text


In [7]:
def collect_docs_to_target_size(streaming_corpora, target_chars_total: int) -> List[str]:
    """
    Stream from multiple datasets and collect cleaned, deduplicated documents
    until we reach approximately `target_chars_total` characters (~1GB).
    Uses SOURCE_FRACTIONS to allocate per-source quotas.
    """
    seen_hashes = set()
    cleaned_docs: List[str] = []

    total_chars_global = 0

    for source_name, fraction in SOURCE_FRACTIONS.items():
        ds = streaming_corpora[source_name]
        target_chars_source = int(target_chars_total * fraction)
        chars_source = 0

        print(f"\nCollecting from {source_name} (target ~{target_chars_source / 1e6:.1f}M chars)...")

        for row in tqdm(ds, desc=f"{source_name} streaming"):
            if source_name == "wikipedia":
                text = row.get("text", "")
            elif source_name == "fineweb":
                text = row.get("text", "")
            else:
                text = row.get("text", "")

            text = clean_text(text)
            if not text:
                continue

            if len(text.split()) < MIN_WORDS:
                continue

            key_str = text[:2000]
            h = hashlib.md5(key_str.encode("utf-8")).hexdigest()
            if h in seen_hashes:
                continue
            seen_hashes.add(h)

            cleaned_docs.append(text)
            chars = len(text)
            chars_source += chars
            total_chars_global += chars

            if chars_source >= target_chars_source:
                print(f"Reached target for {source_name}: {chars_source / 1e6:.1f}M chars.")
                break

        print(f"{source_name}: collected {chars_source / 1e6:.1f}M chars.")

    print(f"\nTOTAL collected chars: {total_chars_global} → ~{total_chars_global / 1e9:.2f} GB")
    return cleaned_docs

cleaned_docs = collect_docs_to_target_size(streaming_corpora, TARGET_CHARS_TOTAL)
len(cleaned_docs)


wikipedia streaming: 133404it [01:22, 1610.28it/s]


Reached target for wikipedia: 600.0M chars.
wikipedia: collected 600.0M chars.



fineweb streaming: 125865it [01:43, 1216.36it/s]


Reached target for fineweb: 600.0M chars.
fineweb: collected 600.0M chars.



ag_news streaming: 120000it [00:16, 7119.22it/s]

ag_news: collected 3.5M chars.

TOTAL collected chars: 1203536778 → ~1.20 GB


262042

In [8]:
def tokenize_and_chunk(docs: List[str], tokenizer, block_size: int) -> List[List[int]]:
    """
    Convert a list of cleaned docs into a big list of token blocks,
    each of length <= block_size.
    """
    token_blocks: List[List[int]] = []

    for doc in tqdm(docs, desc="Tokenizing & chunking"):
        ids = tokenizer.encode(doc, add_special_tokens=False)

        for i in range(0, len(ids), block_size):
            block = ids[i : i + block_size]
            if len(block) == 0:
                continue
            token_blocks.append(block)

    print(f"Total token blocks: {len(token_blocks)}")
    return token_blocks

tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME, use_fast=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

token_blocks = tokenize_and_chunk(cleaned_docs, tokenizer, BLOCK_SIZE)
len(token_blocks)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Tokenizing & chunking: 100%|██████████████████████████████████████████| 262042/262042 [14:07<00:00, 309.24it/s]

Total token blocks: 662834


662834

In [9]:
# Saving for Assignment 2

torch.save(token_blocks, "token_blocks.pt")
tokenizer.save_pretrained("./tokenizer_gpt2")

print("Tokenized dataset saved!")


Tokenized dataset saved!


In [10]:
class TokenBlockDataset(Dataset):
    def __init__(self, token_blocks: List[List[int]]):
        self.token_blocks = token_blocks

    def __len__(self):
        return len(self.token_blocks)

    def __getitem__(self, idx) -> Dict[str, torch.Tensor]:
        ids = self.token_blocks[idx]
        return {
            "input_ids": torch.tensor(ids, dtype=torch.long)
        }


In [11]:
def collate_fn_pad(batch, pad_token_id: int):
    """
    Pads a batch of variable-length input_ids to the same length,
    and creates attention_mask.
    """
    input_ids = [item["input_ids"] for item in batch]
    lengths = [len(x) for x in input_ids]
    max_len = max(lengths)

    padded_ids = []
    attention_masks = []

    for ids in input_ids:
        pad_len = max_len - len(ids)
        if pad_len > 0:
            padded = torch.cat(
                [ids, torch.full((pad_len,), pad_token_id, dtype=torch.long)]
            )
        else:
            padded = ids
        padded_ids.append(padded)
        attention_masks.append((padded != pad_token_id).long())

    return {
        "input_ids": torch.stack(padded_ids, dim=0),
        "attention_mask": torch.stack(attention_masks, dim=0),
    }


In [12]:
token_dataset = TokenBlockDataset(token_blocks)

dataloader = DataLoader(
    token_dataset,
    batch_size=8,
    shuffle=True,
    collate_fn=lambda b: collate_fn_pad(b, tokenizer.pad_token_id),
)

sample_batch = next(iter(dataloader))

print("input_ids shape:", sample_batch["input_ids"].shape)
print("attention_mask shape:", sample_batch["attention_mask"].shape)

torch.save(sample_batch, SAMPLE_PT_PATH)
print(f"Saved sample batch to {SAMPLE_PT_PATH}")


input_ids shape: torch.Size([8, 512])
attention_mask shape: torch.Size([8, 512])
Saved sample batch to ./processed_data/sample_dataset.pt


In [13]:
example_ids = sample_batch["input_ids"][0]
decoded = tokenizer.decode(example_ids.tolist(), skip_special_tokens=True)
decoded[:1000]


'with so many people amazed by how fast we are advancing with the aid of technology, it’s no wonder that so many are interested in how it all works. if you are one of the curious users of the world wide web, read on to find out about the top seven technology trends of 2019-2020. augmented reality & virtual reality virtual reality or vr has been considered the main achievement from the most recent years, but augmented reality or ar is expected to overcome it very soon. there is also the mixed reality or mr to watch out as it seems to be no less promising as the other two technologies. summarized together as r+, these technologies continue to develop and bring new, surprising applications for all kinds of industries. even though r+ was first present primarily in the gaming industry, it is now quickly spreading into manufacturing, space exploration, healthcare, engineering design, and more. both vr and ar are changing the way people work and collaborate with each other from across the wor

Assignment 2

In [14]:
import math
import os
from typing import List, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

SEQ_LEN = 64

EMBED_DIM = 128
NUM_HEADS = 4
NUM_LAYERS = 2
FF_HIDDEN_DIM = 512
DROPOUT = 0.1

BATCH_SIZE = 32
LR = 3e-4
NUM_EPOCHS = 5

CHECKPOINT_PATH = "mini_gpt_checkpoint.pt"


Using device: cuda


In [15]:
# If token_blocks/tokenizer are already in memory from Assignment 1, skip this cell.

if "token_blocks" not in globals():
    print("Loading token_blocks from disk...")
    token_blocks = torch.load("token_blocks.pt")

if "tokenizer" not in globals():
    print("Loading tokenizer from disk...")
    tokenizer = AutoTokenizer.from_pretrained("./tokenizer_gpt2", use_fast=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

vocab_size = len(tokenizer)
print("Number of token blocks:", len(token_blocks))
print("Vocab size:", vocab_size)


Number of token blocks: 662834
Vocab size: 50257


In [16]:
# Flatten token_blocks into one long tensor
all_tokens_list = [tok for block in token_blocks for tok in block]
all_tokens = torch.tensor(all_tokens_list, dtype=torch.long)
print("Total tokens:", len(all_tokens))

class GPTDataset(Dataset):
    """
    Dataset for next-token prediction:
    For index i, returns:
        x = tokens[i : i+SEQ_LEN]
        y = tokens[i+1 : i+SEQ_LEN+1]
    """
    def __init__(self, tokens: torch.Tensor, seq_len: int):
        self.tokens = tokens
        self.seq_len = seq_len

    def __len__(self):
        return len(self.tokens) - self.seq_len - 1

    def __getitem__(self, idx) -> Tuple[torch.Tensor, torch.Tensor]:
        x = self.tokens[idx : idx + self.seq_len]
        y = self.tokens[idx + 1 : idx + 1 + self.seq_len]
        return x, y

dataset = GPTDataset(all_tokens, SEQ_LEN)
len(dataset)


Total tokens: 269811112


269811047

In [17]:
# 90/10 train/val split
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print("Train size:", len(train_dataset))
print("Val size:", len(val_dataset))

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)


Train size: 242829942
Val size: 26981105


In [18]:
def generate_causal_mask(seq_len: int, device: str):
    # shape: (seq_len, seq_len); -inf above diagonal
    mask = torch.triu(torch.full((seq_len, seq_len), float("-inf")), diagonal=1)
    return mask.to(device)


In [19]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model: int, n_heads: int, ff_hidden_dim: int, dropout: float):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = nn.MultiheadAttention(
            embed_dim=d_model,
            num_heads=n_heads,
            dropout=dropout,
            batch_first=True,   # (B, T, C)
        )
        self.ln2 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(
            nn.Linear(d_model, ff_hidden_dim),
            nn.GELU(),
            nn.Linear(ff_hidden_dim, d_model),
            nn.Dropout(dropout),
        )

    def forward(self, x, attn_mask=None):
        # x: (B, T, C)
        h = self.ln1(x)
        attn_out, _ = self.attn(h, h, h, attn_mask=attn_mask)
        x = x + attn_out
        h = self.ln2(x)
        x = x + self.ff(h)
        return x


class MiniGPT(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        d_model: int,
        n_heads: int,
        n_layers: int,
        ff_hidden_dim: int,
        max_seq_len: int,
        dropout: float = 0.1,
    ):
        super().__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.max_seq_len = max_seq_len

        self.token_embed = nn.Embedding(vocab_size, d_model)
        self.pos_embed = nn.Embedding(max_seq_len, d_model)

        self.blocks = nn.ModuleList(
            [
                TransformerBlock(d_model, n_heads, ff_hidden_dim, dropout)
                for _ in range(n_layers)
            ]
        )

        self.ln_f = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        """
        x: (B, T) token IDs
        returns logits: (B, T, vocab_size)
        """
        B, T = x.shape
        if T > self.max_seq_len:
            raise ValueError(f"Sequence length {T} exceeds max_seq_len {self.max_seq_len}")

        # token + positional embeddings
        tok_emb = self.token_embed(x)                      # (B, T, C)
        positions = torch.arange(0, T, device=x.device).unsqueeze(0)  # (1, T)
        pos_emb = self.pos_embed(positions)                # (1, T, C)
        h = tok_emb + pos_emb

        # causal mask: (T, T)
        attn_mask = generate_causal_mask(T, x.device)

        for block in self.blocks:
            h = block(h, attn_mask=attn_mask)

        h = self.ln_f(h)
        logits = self.head(h)                              # (B, T, vocab_size)
        return logits

vocab_size = len(tokenizer)
model = MiniGPT(
    vocab_size=vocab_size,
    d_model=EMBED_DIM,
    n_heads=NUM_HEADS,
    n_layers=NUM_LAYERS,
    ff_hidden_dim=FF_HIDDEN_DIM,
    max_seq_len=SEQ_LEN,
    dropout=DROPOUT,
).to(DEVICE)

sum_params = sum(p.numel() for p in model.parameters())
print(model)
print("Total parameters:", sum_params)


MiniGPT(
  (token_embed): Embedding(50257, 128)
  (pos_embed): Embedding(64, 128)
  (blocks): ModuleList(
    (0-1): 2 x TransformerBlock(
      (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
      )
      (ln2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (ff): Sequential(
        (0): Linear(in_features=128, out_features=512, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=512, out_features=128, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (head): Linear(in_features=128, out_features=50257, bias=True)
)
Total parameters: 13321041


In [20]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

def compute_loss_and_ppl(logits: torch.Tensor, targets: torch.Tensor):
    """
    logits: (B, T, V)
    targets: (B, T)
    """
    B, T, V = logits.shape
    loss = F.cross_entropy(
        logits.view(B * T, V),
        targets.view(B * T),
        reduction="mean",
    )
    ppl = math.exp(loss.item())
    return loss, ppl


In [21]:
def train_one_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0.0
    total_batches = 0

    for x, y in dataloader:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        logits = model(x)
        loss, _ = compute_loss_and_ppl(logits, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_batches += 1

    avg_loss = total_loss / max(total_batches, 1)
    avg_ppl = math.exp(avg_loss)
    return avg_loss, avg_ppl


def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0.0
    total_batches = 0

    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            y = y.to(device)
            logits = model(x)
            loss, _ = compute_loss_and_ppl(logits, y)

            total_loss += loss.item()
            total_batches += 1

    avg_loss = total_loss / max(total_batches, 1)
    avg_ppl = math.exp(avg_loss)
    return avg_loss, avg_ppl


In [22]:
train_losses = []
val_losses = []
train_ppls = []
val_ppls = []

best_val_loss = float("inf")

for epoch in range(1, NUM_EPOCHS + 1):
    train_loss, train_ppl = train_one_epoch(model, train_loader, optimizer, DEVICE)
    val_loss, val_ppl = evaluate(model, val_loader, DEVICE)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_ppls.append(train_ppl)
    val_ppls.append(val_ppl)

    print(f"Epoch {epoch}/{NUM_EPOCHS} "
          f"- train loss: {train_loss:.4f}, train ppl: {train_ppl:.2f} "
          f"- val loss: {val_loss:.4f}, val ppl: {val_ppl:.2f}")

    # Save best checkpoint
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(
            {
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "config": {
                    "vocab_size": vocab_size,
                    "d_model": EMBED_DIM,
                    "n_heads": NUM_HEADS,
                    "n_layers": NUM_LAYERS,
                    "ff_hidden_dim": FF_HIDDEN_DIM,
                    "seq_len": SEQ_LEN,
                    "dropout": DROPOUT,
                    "lr": LR,
                    "batch_size": BATCH_SIZE,
                    "num_epochs": NUM_EPOCHS,
                },
            },
            CHECKPOINT_PATH,
        )
        print(f"New best model saved to {CHECKPOINT_PATH}")


In [23]:
epochs = range(1, NUM_EPOCHS + 1)

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label="Train Loss")
plt.plot(epochs, val_losses, label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training & Validation Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, train_ppls, label="Train Perplexity")
plt.plot(epochs, val_ppls, label="Val Perplexity")
plt.xlabel("Epoch")
plt.ylabel("Perplexity")
plt.title("Training & Validation Perplexity")
plt.legend()

plt.tight_layout()
plt.show()
